In [2]:
from langchain_ollama import OllamaEmbeddings,ChatOllama
embedding_model=OllamaEmbeddings(model="nomic-embed-text:latest")
llm=ChatOllama(model="phi4:latest")

In [ ]:
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain